This notebook is meant to download data from copernicus CMIP6.

Data source : https://cds.climate.copernicus.eu/cdsapp#!/dataset/projections-cmip6?tab=form

# User input

In [1]:
period_of_interest = 'future' # 'future' (2015-2100) or 'past' (1950-2014)
global_variable = 'temperature'
name_variable = 'daily_maximum_near_surface_air_temperature' 

# 'tas' 'near_surface_air_temperature'
# 'tasmax' 'daily_maximum_near_surface_air_temperature'
# 'tasmin' 'daily_minimum_near_surface_air_temperature'
# 'hurs' 'near_surface_specific_humidity'
# 'Wind' 'near_surface_wind_speed'

temporal_resolution = 'daily'

# wind register at 10 m

# Functions and packages

In [2]:
import pandas as pd
import numpy as np
import numpy.ma as ma
import netCDF4 as nc#not directly used but needs to be imported for some nc4 files manipulations, use for nc files
from netCDF4 import Dataset
import xarray as xr
import datetime # to have actual date
import os
import os.path
import cdsapi # for copernicus function
import shutil
from datetime import datetime

# Out path

In [3]:
out_path=r'\\COWI.net\projects\A245000\A248363\CRVA\Datasets'

# Area information

This code download data one area at the time. The user must indicate which part of the word he/she wnats to download.

Reminder: 

latitude is vertical, it specifies North-South positions.

longitude is horizontal, it specifies West-East positions.

In [4]:
# default coordinates are the ones of Mozambique

name_area = 'all-Mozambique'

North = -10
South = -27
West = 30
East = 41

area = [North, West, South, East] # list format

# Class

### Calendar class

In [5]:
# class to define parameter of time that remain constant durinf the whole script
class calendar:
    default_month = [ 
                '01', '02', '03',
                '04', '05', '06',
                '07', '08', '09',
                '10', '11', '12',
                ]
    default_day = [
                '01', '02', '03',
                '04', '05', '06',
                '07', '08', '09',
                '10', '11', '12',
                '13', '14', '15',
                '16', '17', '18',
                '19', '20', '21',
                '22', '23', '24',
                '25', '26', '27',
                '28', '29', '30',
                '31',
                ]
    #actual_date = datetime.date.today()
    #actual_year = actual_date.year

### Copernicus class

In [6]:
## Definition of tuples that will be useful to search which data are available or not
# make it tuples to make unchangeable
class copernicus_elements:
    # there is 58 models
    models =('access_cm2','awi_cm_1_1_mr','bcc_csm2_mr','cams_csm1_0','canesm5_canoe','cesm2_fv2','cesm2_waccm_fv2','cmcc_cm2_hr4','cmcc_esm2','cnrm_cm6_1_hr','e3sm_1_0','e3sm_1_1_eca','ec_earth3_aerchem','ec_earth3_veg','fgoals_f3_l','fio_esm_2_0','giss_e2_1_g','hadgem3_gc31_ll','iitm_esm','inm_cm5_0','ipsl_cm6a_lr','kiost_esm','miroc6','miroc_es2l','mpi_esm1_2_hr','mri_esm2_0','norcpm1','noresm2_mm','taiesm1','access_esm1_5','awi_esm_1_1_lr','bcc_esm1','canesm5','cesm2','cesm2_waccm','ciesm','cmcc_cm2_sr5','cnrm_cm6_1','cnrm_esm2_1','e3sm_1_1','ec_earth3','ec_earth3_cc','ec_earth3_veg_lr','fgoals_g3','gfdl_esm4','giss_e2_1_h','hadgem3_gc31_mm','inm_cm4_8','ipsl_cm5a2_inca','kace_1_0_g','mcm_ua_1_0','miroc_es2h','mpi_esm_1_2_ham','mpi_esm1_2_lr','nesm3','noresm2_lm','sam0_unicon','ukesm1_0_ll')
    experiments = ('ssp1_1_9','ssp1_2_6','ssp4_3_4','ssp5_3_4os','ssp2_4_5','ssp4_6_0','ssp3_7_0','ssp5_8_5')
    experiments_historical=('historical',)

In [7]:
if period_of_interest == 'past':
    y_start = 1950
    y_end = 2014
    scenarios = copernicus_elements.experiments_historical
if period_of_interest == 'future':
    y_start = 2015
    y_end = 2100
    scenarios = copernicus_elements.experiments

## Functions

In [8]:
################################################ Period for copernicus function ################################################
# Aim of the function: by giving it a first and last year of the period that must analyzed, this function produce several 
# vectors,containing time informations, useful to download and treat data from CMIP6 projections (https://cds.climate.copernicus.eu/cdsapp#!/dataset/projections-cmip6?tab=overview )
# Those time vectors are used in the copernicus_data and the dataframe_copernicus and csv_copernicus functions

# function year_copernicus produce 
# year: a vector containing all the year in the period of interest
# year_str: an array containing all the year in the period of interest in the string format
# index: an array containing the index of the year and year_str
#### Parameters of the function
# first_year: number in int format, of the first year of the period of interest
# last_year: number in int format, of the last year of the period of interest
def year_copernicus(first_year,last_year):
    year = np.arange(first_year,(last_year+1),1) # create vector of years
    year_str = [0]*len(year) # create initiale empty vector to convert years in int
    index = np.arange(0,len(year)) # create vector of index for year
    i = 0 # initialize index
    for i in index: # convert all the date in string format
        year_str[i]=str(year[i])
    return (year, year_str, index)

# function date_copernicus produce 
# dates: the format depend on the temporal reolution, but always contain the dates of the period of interest.
#        with temporal_resolution=daily, dates is a DatetimeIndex
#        with temporal_resolution=monthly, dates is a list
# index_dates: an array containing the index of the dates
#### Parameters of the function
# temporal_resolution: daily or monthly
# year_str: ???? produce by function year_copernicus, containing the year of the period of interest in string format
def date_copernicus(temporal_resolution,year_str):
    start_date = "01-01-"+year_str[0] # string start date based on start year
    stop_date = "31-12-"+year_str[len(year_str)-1] # string stop date based on stop year
    if temporal_resolution =='daily':
        # vector of dates between start date and stop date
        dates = pd.date_range(start_date,stop_date)# dates is a pandas.core.indexes.datetimes.DatetimeIndex
        # By default, freq = 'D', which means calendar day frequency (source : https://pandas.pydata.org/docs/user_guide/timeseries.html#timeseries-offset-aliases)
        #index_dates = np.arange(0,len(dates)) # vector containning index o dates vector
    if temporal_resolution =='monthly':
        dates = pd.date_range(start_date,stop_date,freq='MS') # vector of dates between start date and stop date
        dates=list(dates.strftime('%m-%Y')) # dates is an pandas.core.indexes.base.Index, not a pandas.core.indexes.datetimes.DatetimeIndex
    #if temporal_resolution =='fixed': trouver donnees pour gerer cela
    index_dates = np.arange(0,len(dates)) # vector containning index o dates vector
    return (dates, index_dates)

In [9]:
########################################### Download data from Copernicus Website ############################################
# Aim of the function: this function aims to downloaded with the function copernicus_data the files of interest from the website
# Copernicus CMIP6
# Actions of this function
#     1) Create the string indicating the period of interest
#     2) Download data, with its corresponding experiments and models
#        2 a) Creates path for file
#        2 b) Thanks to function 'copernicus_data', download the data inthe path just created

# Parameters of the function
# temporal_resolution: 'daily', 'monthly', or 'fixed'. String type 
# year_str: list containing all the years under the string type and in the period of interest
# experiments: copernicus_elements.experiments
# models: copernicus_elements.models
# out_path: path were the outputs are registered. Defined by the user at the beginning of the code 
# global_variable: global name of the climate variable of interest (example: Wind)
# name_variable: name of the elements downloaded from copernicus (example: 'near_surface_wind_speed')
# area: list containing latitudes and longitudes of area of interest
# name_area: Name of the area of interest
# source: name of the source of the data, by default it is 'Copernicus-CMIP6'

def download_copernicus(temporal_resolution,year_str,experiments,models,out_path, global_variable, name_variable,area, name_area,source='Copernicus-CMIP6'):    
    # create string for name of folder depending on type of period
    if temporal_resolution == 'fixed':
        period = 'fixed'
    else:
        period=year_str[0]+'-'+year_str[len(year_str)-1]
        
    for SSP in experiments:
        experiment = (SSP,) # create tuple for iteration of dataframe
        for model_simulation in models:
            model =(model_simulation,) # create tuple for iteration of dataframe
            print('Test with scenario '+SSP+' and model '+model_simulation)
            # path were the futur downloaded file is registered
            path_for_file= os.path.join(out_path,global_variable,name_variable,source,'raw_data',period,SSP,model_simulation)
            #path_for_file = path_length(path_for_file)
            # existence of path_for_file tested in copernicus function
            copernicus_data(temporal_resolution,SSP,name_variable,model_simulation,year_str,area,path_for_file,out_path,name_area,source)

    return

In [10]:
################################################### Copernicus data function ###################################################
# Aim of the function : Determine if the data were already downloadeed or not. If the data were not downloaded, call
# function 'try_download_copernicus' to download them. If the data were already downloaded but not extracted, call function
# 'download_extract'
# Actions of this function
#     1) create the string of the path where the data will be registered 
#        
#     2) Check if the path where the data will be registered already exists or not
#        2 a) The path where the data will be registered does not exist
#             The path where are the compressed data exists ?
#                   ---> no : call the 'try_download_copernicus' to download and extract the data of interest
#                   ---> yes : is the path empty ?
#                              > yes : use 'try_download_copernicus' to download and extract the data of interest
#                              > no : call function 'download_extract' to extract the files in this path to path_for_file
#        2 b) The path where the data will be registered does exist
#             Is the path empty ?
#                   ---> yes : the path where the compressed data are exists ?
#                             > no : use 'try_download_copernicus' to download and extract the data of interest
#                             > yes : is the folder empty ?
#                                     * yes : use 'try_download_copernicus' to download and extract the data of interest
#                                     * no : call function 'download_extract' to extract the files in this path to path_for_file
#                   ---> no : the data were downloaded and extracted for the elements of interest 

#### Parameters of the function
# temporal_resolution : daily or monthly or fixed
# SSP : scenario that is studied "Historical", "SSP1-1.9", "SSP1-2.6" ...
# name_variable : variable to be studied
# model: model of projection to choose
# year: year(s) of study to choose
# area: area of study, if not specific, area should be an empty array area=[]
# path_for_file: path where the file must be unzipped
# out_path: path were all the outputs are registered, defined by the user in the begining of the main code
# name_area :  Name of the area of interest
# source: name of the source of the data (here 'Copernicus-CMIP6')

def copernicus_data(temporal_resolution,SSP,name_variable,model,year,area,path_for_file,out_path,name_area,source): 
    # create path for the downloaded file
    start_path = os.path.join(out_path,'Data_download_zip')
    file_download=create_file_download_path(start_path,name_variable,name_area,SSP,model,year,temporal_resolution,source)
    
    if not os.path.isdir(path_for_file):
        # the path for the file does not exist
        print('path_for_file does not exist: the data may not have been downloaded') 
        if not os.path.isdir(file_download):
            print('file_download does not exist: the data were not downloaded')
            # function try to download from copernicus
            try_download_copernicus(temporal_resolution,SSP,name_variable,model,area,year,path_for_file,file_download,name_area)
            return
            
        else: # if the path already exist, the data in zip format should also exists
            if not os.path.isfile(os.path.join(file_download,'download.zip')):
                print('The path for the download file exists, but is empty')
                # function try to download from copernicus
                try_download_copernicus(temporal_resolution,SSP,name_variable,model,area,year,path_for_file,file_download,name_area)                
                return
            else:
                print('file_download does exist, the data have been downloaded, but not extracted')
                download_extract(path_for_file,file_download,name_area)
                return
                
    else: 
        # the path for the file exists
        if not os.listdir(path_for_file): # if the path is empty
            if not os.path.isdir(file_download):
                print('file_download does not exist: the data were not downloaded')
                # function try to download from copernicus
                try_download_copernicus(temporal_resolution,SSP,name_variable,model,area,year,path_for_file,file_download,name_area)
                return

            else: # if the path already exist, the data in zip format should also exists
                if not os.path.isfile(os.path.join(file_download,'download.zip')):
                    print('The path for the download file exists, but is empty')
                    # function try to download from copernicus
                    try_download_copernicus(temporal_resolution,SSP,name_variable,model,area,year,path_for_file,file_download,name_area)
                    return
                else:
                    print('file_download does exist, the data have been downloaded, but not extracted')
                    download_extract(path_for_file,file_download,name_area)
                    return
        
        else: # if the path is not empty
            print(f'The data has already been downloaded and extracted at {path_for_file}')
            return

In [11]:
################################################### try_download_copernicus function ###################################################
# Aim of the function : download compressed files from the Copernicus CMIP6 website (indicated at the beginning of the website)

##### Actions of the functions :
#     1) import function cdsapi.Client(). Will be use in step 3, with c.retrieve
#     2) Create the variables dictionnary, depending on the parameters of 'try_download_copernicus'. this dictionnary will then
#        be used in the c.retrieve function
#     3) Try to downloaded compressed file from Copernicus CMIP6 website. If parameters are not matching, will fall in except 
#        and continue
#     4) Use function 'download_extract' to extract compressed files in path_for_file

##### Parameters :
# temporal_resolution : daily or monthly or fixed
# SSP : scenario that is studied "Historical", "SSP1-1.9", "SSP1-2.6" ...
# name_variable : variable to be studied
# model: model of projection to choose
# area: area of study, if not specific, area should be an empty array area=[]
# year: year(s) of study to choose
# path_for_file: path where the file must be unzipped
# file_download : path where the compressed files under a zip format are registered
# name_area : Name of the area of interest

def try_download_copernicus(temporal_resolution,SSP,name_variable,model,area,year,path_for_file,file_download,name_area):
    c = cdsapi.Client()# function to use the c.retrieve
    # basic needed dictionnary to give to the c.retrieve function the parameters asked by the user
    variables = {
                'format': 'zip', # this function is only designed to download and unzip zip files
                'temporal_resolution': temporal_resolution,
                'experiment': SSP,
                'variable': name_variable,
                'model': model,
    }

    if area != []: # the user is interested by a sub region and not the whole region 
        variables.update({'area':area}) 

    if (name_variable=='air_temperature') or (name_variable=='relative_humidity') or (name_variable=='specific_humidity') or (name_variable=='eastward_wind') or (name_variable=='northward_wind') or (name_variable=='geopotential_height'): # test if variable is temperature
        variables['level'] = '1000' # [hPa], value of the standard pressure at sea level is 1013.25 [hPa], so 1000 [hPa] is the neareste value. Other pressure value are available but there is no interest for the aim of this project

    if temporal_resolution != 'fixed':# if 'fixed', no year, month, date to choose
        variables['year']=year # period chosen by the user
        variables['month']= calendar.default_month  # be default, all the months are given; defined in class calendar
        if temporal_resolution == 'daily':
            variables['day']= calendar.default_day # be default, all the days are given; defined in class calendar
    # c.retrieve download the data from the website
    try:
        c.retrieve(
            'projections-cmip6',
            variables,
            'download.zip') # the file in a zip format is registered in the current directory
    except:
        print('Some parameters are not matching')
        return # stop the function, because some data the user entered are not matching
    print('The file has been download')
    # function to extract the downloaded zip
    download_extract(path_for_file,file_download,name_area)
    print('The file has been extracted')
    return

In [12]:
# 'download_extract' function aims to extract in path_for_file, the downloaded file in zip format which is registered 
# in file_download

#### Actions of the function :
#     1) Check if the path_for_file, where the decompressed file should be registered, exists
#        > no : ensure the creation of the path with os.makedirs
#     2) Check if the compressed file is in the working directory
#        > no : move the compressed fil to the working directory
#     3) Extract the compressed file, in a folder named 'name_area'
#     4) Check if the file_download, where the compressed file should be registered, exists
#        > no : ensure the creation of the path with os.makedirs
#     5) move the files to the appropriate places

def download_extract(path_for_file,file_download,name_area):
    # step 1
    path_for_file = os.path.join(path_for_file,name_area)
    if not os.path.isdir(path_for_file): # path_for_file does not exists, need to ensure that is is created
        os.makedirs(path_for_file) # to ensure the creation of the path
        print(path_for_file)
        print('Path for the file is created, did not existed before')
    
    # step 2
    if 'download.zip' not in os.listdir(): # check if download is in the working directory
        print('The download zip is moved to the working directory')
        path_downloaded_zip=os.path.join(file_download,'download.zip')
        shutil.move(path_downloaded_zip,r'C:\Users\CLMRX\OneDrive - COWI\Documents\GitHub\CRVA_tool') # move download file to working directory
    
    # step 3
    from zipfile import ZipFile
    zf = ZipFile('download.zip', 'r')
    zf.extractall(name_area) # if no precision of directory, extract in current directory
    zf.close()

    # step 4
    if not os.path.isdir(file_download): # path_for_file does not exists, need to ensure that is is created
        os.makedirs(file_download) # to ensure the creation of the path
    
    # step 5
    # moving download to appropriate place
    shutil.move('download.zip',file_download) # no need to delete 'download.zip' from inital place
    for file in os.listdir(name_area):
        try:
            shutil.move(os.path.join(r'C:\Users\CLMRX\OneDrive - COWI\Documents\GitHub\CRVA_tool',name_area,file),os.path.join(path_for_file,file)) # move extracted data to the path created for them
            print('the file named '+file+ ' was moved')
        except:
            #os.remove(file) # does not work
            pass
    #if name_area in os.listdir():
    #    print(name_area + ' is here')
    #    shutil.rmtree(name_area)
    #    print(name_area + ' has been deleted')
    print('\n The downloaded file is extracted')
    return

In [13]:
# 'create_file_download_path' function to create path for the downloaded compressed folder

def create_file_download_path(start_path,name_variable,name_area,SSP,model,year,temporal_resolution,source):
    # adapt the name of the folder for the period, depending on the type of period
    if len(year)==1:
        file_download = os.path.join(start_path,name_variable,source,year,SSP,model,name_area)
    elif len(year)>1:
        period=year[0]+'-'+year[len(year)-1]
        file_download = os.path.join(start_path,name_variable,source,period,SSP,model,name_area)
    elif temporal_resolution == 'fixed':
        file_download = os.path.join(start_path,name_variable,source,'fixed_period',SSP,model,name_area)
    #file_download = path_length(file_download)
    return file_download # returns string of the path where the downloaded compressed folder must be registered

In [14]:
# seach_for_nc is a function looking in path_for_file for a document in .nc format

def search_for_nc(path_for_file):
    print('path_for_file does exist Function copernicus search for nc')
    for file in os.listdir(path_for_file):
        if file.endswith(".nc"):
            final_path=os.path.join(path_for_file, file)
            
            print('The file is in the path Function copernicus search for nc\n')
            print('Before path_length, The final path for the nc file is: '+final_path)
            answer = str(os.path.isfile(final_path))
            print('\n The final path for nc file exists ? '+answer+'\n')
            final_path=path_length(final_path) # check if length of path is too long
            print('After path_length, The final path for the nc file is: '+final_path)
            answer = str(os.path.isfile(final_path))
            print('\n The final path for nc file exists ? '+answer+'\n')
            return final_path # the function returns the path of the nc file of interest
            break # stop the function if a nc file was found 
        else:
            pass
    # the all folder has been search and there is no nc file in it
    print('Problem : No nc file was found Function copernicus Function copernicus search for nc')# this line is out of the for loop, 
    #because it should only appear once all the folder has been examinated and if the break of the if was not used

In [15]:
# this functions test if the path is too long
# if the path is more than 250 char, the path wll be modified in order for windows to accept is as a path

def path_length(str1):
    if len(str1)>250:
        path = os.path.abspath(str1) # normalize path
        if path.startswith(u"\\\\"):
            path=u"\\\\?\\UNC\\"+path[2:]
        else:
            path=u"\\\\?\\"+path
        return path
    else:
        return str1

# Register copernicus data

In [16]:
(year, year_str, index)=year_copernicus(y_start,y_end)

In [ ]:
download_copernicus(temporal_resolution,year_str,scenarios,copernicus_elements.models,out_path, global_variable, name_variable,area,name_area)

Test with scenario ssp1_1_9 and model access_cm2
path_for_file does not exist: the data may not have been downloaded
file_download does not exist: the data were not downloaded


2023-07-14 15:09:03,631 INFO Welcome to the CDS
2023-07-14 15:09:03,632 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/projections-cmip6
2023-07-14 15:09:03,677 INFO Request is queued
2023-07-14 15:09:04,725 INFO Request is failed
2023-07-14 15:09:04,726 ERROR Message: an internal error occurred processing your request
2023-07-14 15:09:04,727 ERROR Reason:  No matching data for request {'experiment': 'ssp119', 'model': 'ACCESS-CM2', 'temporal_resolution': 'day', 'variable': 'tasmax'}
2023-07-14 15:09:04,727 ERROR   Traceback (most recent call last):
2023-07-14 15:09:04,728 ERROR     File "/opt/cdstoolbox/cdscompute/cdscompute/cdshandlers/services/handler.py", line 59, in handle_request
2023-07-14 15:09:04,729 ERROR       result = cached(context.method, proc, context, context.args, context.kwargs)
2023-07-14 15:09:04,729 ERROR     File "/opt/cdstoolbox/cdscompute/cdscompute/caching.py", line 108, in cached
2023-07-14 15:09:04,730 ERROR       result = proc(con

Some parameters are not matching
Test with scenario ssp1_1_9 and model awi_cm_1_1_mr
path_for_file does not exist: the data may not have been downloaded
file_download does not exist: the data were not downloaded


2023-07-14 15:09:05,872 INFO Request is failed
2023-07-14 15:09:05,873 ERROR Message: an internal error occurred processing your request
2023-07-14 15:09:05,874 ERROR Reason:  No matching data for request {'experiment': 'ssp119', 'model': 'AWI-CM-1-1-MR', 'temporal_resolution': 'day', 'variable': 'tasmax'}
2023-07-14 15:09:05,874 ERROR   Traceback (most recent call last):
2023-07-14 15:09:05,875 ERROR     File "/opt/cdstoolbox/cdscompute/cdscompute/cdshandlers/services/handler.py", line 59, in handle_request
2023-07-14 15:09:05,876 ERROR       result = cached(context.method, proc, context, context.args, context.kwargs)
2023-07-14 15:09:05,877 ERROR     File "/opt/cdstoolbox/cdscompute/cdscompute/caching.py", line 108, in cached
2023-07-14 15:09:05,877 ERROR       result = proc(context, *context.args, **context.kwargs)
2023-07-14 15:09:05,878 ERROR     File "/opt/cdstoolbox/cdscompute/cdscompute/services.py", line 124, in __call__
2023-07-14 15:09:05,878 ERROR       return p(*args, **kw

Some parameters are not matching
Test with scenario ssp1_1_9 and model bcc_csm2_mr
path_for_file does not exist: the data may not have been downloaded
file_download does not exist: the data were not downloaded


...

2023-07-14 17:37:44,636 INFO Welcome to the CDS
2023-07-14 17:37:44,637 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/projections-cmip6
2023-07-14 17:37:44,707 INFO Request is queued


the file named tasmax_day_INM-CM5-0_ssp585_r1i1p1f1_gr1_20150101-21001231_v20190724.nc was moved

 The downloaded file is extracted
The file has been extracted
Test with scenario ssp5_8_5 and model ipsl_cm6a_lr
path_for_file does not exist: the data may not have been downloaded
file_download does not exist: the data were not downloaded


2023-07-14 17:37:45,749 INFO Request is failed
2023-07-14 17:37:45,750 ERROR Message: an internal error occurred processing your request
2023-07-14 17:37:45,750 ERROR Reason:  No matching data for request {'experiment': 'ssp585', 'model': 'IPSL-CM6A-LR', 'temporal_resolution': 'day', 'variable': 'tasmax'}
2023-07-14 17:37:45,751 ERROR   Traceback (most recent call last):
2023-07-14 17:37:45,751 ERROR     File "/opt/cdstoolbox/cdscompute/cdscompute/cdshandlers/services/handler.py", line 59, in handle_request
2023-07-14 17:37:45,752 ERROR       result = cached(context.method, proc, context, context.args, context.kwargs)
2023-07-14 17:37:45,753 ERROR     File "/opt/cdstoolbox/cdscompute/cdscompute/caching.py", line 108, in cached
2023-07-14 17:37:45,754 ERROR       result = proc(context, *context.args, **context.kwargs)
2023-07-14 17:37:45,754 ERROR     File "/opt/cdstoolbox/cdscompute/cdscompute/services.py", line 124, in __call__
2023-07-14 17:37:45,755 ERROR       return p(*args, **kwa

Some parameters are not matching
Test with scenario ssp5_8_5 and model kiost_esm
path_for_file does not exist: the data may not have been downloaded
file_download does not exist: the data were not downloaded


2023-07-14 17:37:46,915 INFO Request is running
